### Custom variable pattern

In [4]:
import gsconfig

# Шаблон задан как строка
template_body = """
- {% substring %} внутри строки
- Строка подстрока в кавычках: "{% variable %}"
- Явное указание на строку: {% substring!string %}
- Сколько коров?
- Вот столько: {% cow!int %} (int) или {% cow %} (source) 
- Вот int который как float {% int!float %} (float)
- И еще давай словарик
{# Я комментарий внутри шаблона #}

- Вот словарик: {% dict %}
{% if check %}
- Тут проверка включать строку или нет ({% check %})
{% endif %}
{% if nope %}
- А эту строку мы не включаем в результат!
{% endif %}
- И пустота: "{% empty %}"
- Тут я беру значение из списка по индексу и обьявляю его строкой: {% list!get_2!string %}
{# Вообще команд может быть любое количество если это имеет смысл #}
- Разворачивая список из лишнего списка: {% list_in_list!extract %}
- Тут список не извлекаю из лишнего списка: {% list_in_list %}
"""

# Класс шаблона c указанием тела шаблона и паттерна
template = gsconfig.Template(body=template_body)

# Словарь с переменными для замены
data = {
    'check': True,
    'nope': False,
    'variable': 'Woohooo!',
    'substring': 'subSTRing',
    'cow': 10.9,
    'int': 10,
    'dict': {'key': 'value', 'list': [{'one': 'odin'}, 'two', ['three', 'four', 'five']]},
    'empty': '',
    'list_in_list': [['Zero', 'One', 'Two', 'Three', 'Four']],
    'list': ['Zero', 'One', 'Two', 'Three', 'Four']
}

# Собрать результат из шаблона и данных
r = template.make(data)
print(r)



- subSTRing внутри строки
- Строка подстрока в кавычках: "Woohooo!"
- Явное указание на строку: "subSTRing"
- Сколько коров?
- Вот столько: 10 (int) или 10.9 (source) 
- Вот int который как float 10.0 (float)
- И еще давай словарик

- Вот словарик: {"key": "value", "list": [{"one": "odin"}, "two", ["three", "four", "five"]]}
- Тут проверка включать строку или нет (true)
- И пустота: ""
- Тут я беру значение из списка по индексу и обьявляю его строкой: "Two"
- Разворачивая список из лишнего списка: ["Zero", "One", "Two", "Three", "Four"]
- Тут список не извлекаю из лишнего списка: [["Zero", "One", "Two", "Three", "Four"]]



### Default variable pattern + template file

In [1]:
import gsconfig
import json

template_path = 'template.tpl'
template = gsconfig.Template(path=template_path, jsonify=True)  # jsonify - пытается перевести результат в JSON

# Словарь с переменными для замены
data = {
    'variable': 'Woohooo!',
    'substring': 'subSTRing',
    'cow': 10.9,
    'dict': {'key': 'value', 'list': [{'one': 'odin'}, 'two', ['three', 'four', 'five']]},
    'empty': ''
}

# r = json.loads(template.make(data))
r = template.make(data)
print(r)



{'string': 'Woohooo!', 'string_with_command': 'Woohooo!', 'string_as_substring': 'Звучит как subSTRing. Похоже?', 'int_as_substring': 'Звучит как 10.9?', 'digit_int': 10, 'digit_source': 10.9, 'dict': {'key': 'value', 'list': [{'one': 'odin'}, 'two', ['three', 'four', 'five']]}, 'empty': ''}


### Дополнительные команды доступные в шаблоне

In [ ]:
"""
Дополнительные команды доступные в ключах шаблона:
dummy -- Пустышка, ничего не длает.

float -- Переводит в начения с плавающей запятой.
Пример: Получено число 10, в шаблон оно будет записано как 10.0

int -- Переводит в целые значения отбрасыванием дробной части
Пример: Получено число 10.9, в шаблон оно будет записано как 10

extract -- Вытаскивает элемент из списка (list or tuple) если это список единичной длины.
Пример: По умолчанию парсер v1 не разворачивает словари и они приходят вида [{'a': 1, 'b': 2}],
если обязательно нужен словарь, то extract развернёт полученный список до {'a': 1, 'b': 2}

wrap -- Дополнительно заворачивает полученый список если первый элемент этого списка не является списком.
Пример: Получен список [1, 2, 4], "1" - первый элемент, это не список, тогда он дополнительно будет завернут [[1, 2, 4]].

string -- Дополнительно заворачивает строку в кавычки. Все прочие типы данных оставляет как есть. 
Используется когда заранее неизвестно будет ли там значение и выбор между null и строкой.
Например, в новостях мультиивентов поле "sns": {news_sns!string}.
Пример: Получена строка 'one,two,three', тогда она будет завернута в кавычки и станет '"one,two,three"'.
"""